# Data wrangling on OpenStreetMaps data
-------------
In this project -- which is part of the Udacity Data Analysis Nanodegree -- I will apply some data munging techniques, such as assessing the quality of the data for validity, accuracy, completeness, consistency and uniformity, to clean an specific area from OpenStreetMap data. After it, in order to try database manipulation in Python, I will load the cleaned data to a MongoDB collection (installed locally in my machine) and apply some simple statistics on it.  

*This Jupyter notebook was iterated over the whole data wrangling process. This final version was also cleaned to offer a project overview in a thorough but also succint way. If you are curious about how it all was build, take a look at the different versions on the Git repository.*

## Choosing an OpenStreetMap region: Missões!
My region of interest in this project is Santo Ângelo, a countryside small city in the southest estate of Brazil which were my birthplace. However, since there's few data for this city and I'm supposed to deal with databases larger than 50MB, for this project, I will consider all the neighboring cities, which in turn constitute the "Missões" region [1] and represent an important chapter in the South American history, since the first settlements were founded during the Spanish colonial missions [2].  

<img src="https://lh3.googleusercontent.com/oGx4LKGq3sWb7Xkf4fsGBnxankZenzudDpU1I7nKIZxH7nT8OVV5COhb8XgchKHpk4GoveTM3B_dHU3tPtUTbo3SrjR19n45kxli6GPnGgHFM3O6qe68wBQVnQKePtnIBa_HX473yWggGuE8sTZaoEQOX4hHehMyZnp2ldOxji79vN6lQSMuDziFKz2NK-E9TASvEDnPJtVaxnuGHzWrjMfjd3bjF32PkZWKqmQjOouU1eCSOW2KM5gb53ntQYvAZT7K6TbaPP_rhwtrJGiX9BAoAtUMo6jA2CpfygbK9RMsstGgiF2IZRxVRwtOeCYLCIaJp6O6Bxx_1QeVIKZiPl4IB2xQWHJU9DHrFI-BGTEatLdyNRW-w9vkn_-erwalDNAHX-GzzxNKAXg6ezI7dUxe-jB2s1as4eIPi3dZibmEuzfQ3wSmmMm3irptNMQGwGvsp67Q92HqZFrf5K9gm9Z3OywTy9X2IRSnfkzVtJsNmU1aH8IAyAPbPq3lJST2bueFKnt6yFxbCFz0Pv1w15NxaQ6ZT24iRhWkbutkLRKOn5s6XLXXucKAR-AwB-ehIKhusiQ5elJK2Hl6EeseEawH8_8b5NCKQ4eiIhDi76BpqcQfVJCcltEQXNzBEzPBrcdYsiDG2Qm0MSWvQXbuC6Fbno7gURB3RXMQhv1l8-eS4np29VrB8Ehi=s1043-no" alt="missoes" width="600"/>

Although today there are 46 municipalities composing this region, in the early eighteenth century there were only 7 villages, nowadays known in Portuguese as the "Sete Povos das Missões".  

For this project, though, I have the following objectives:  
i. Collect, audit and clean data from OpenStreetMap related to this region;  
ii. Get insights from data regarding this region historical value, as well as its current touristic status.

-------------
## Some basic statements

In [94]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import pprint
import re
import codecs
import json

# Dataset file name:
FILENAME = 'Missoes.osm'

-------------
## Getting and reading the data:
The data was obtained from OpenStreetMap (OSM) selecting a specific region in the southern part of Brazil, as shown below:  

<img src="OSM_missoes_region.png" alt="missoes" width="600"/>

After downloading the data, the first step I should do if I did not know the data model would be a simple "less" shell command to figure out what kind of data were in it. Since OpenStreetMaps provides us with a data model, which in turn tells us how the information is organized inside the database, we get to know that the information we are interested in are in keys called 'tag'. Just to check how many of them we will have to process on this data:

In [102]:
#%% Getting acquainted to the dataset
def count_tags(filename):
    tags = {}
    for event, element in ET.iterparse(filename):
        if element.tag not in tags:
            tags[element.tag] = 1
        else: 
            tags[element.tag] += 1
    return tags

tags = count_tags(FILENAME)
pprint.pprint(tags)

{'bounds': 1,
 'member': 15589,
 'meta': 1,
 'nd': 444388,
 'node': 380322,
 'note': 1,
 'osm': 1,
 'relation': 949,
 'tag': 93511,
 'way': 45320}


There are 93.511 tags we'll be dealing with in the next steps. We can move forward to the next step: starting to audit our data. 

-------------
## Auditing data:  
The auditing questions comes when we start exploring the data or, if it's the case we have a prior knowledge of the problem, we already have in mind some issues to investigate. Considering there are available on Internet some similar analysis on OpenStreetMap data [3,4]; and also considering my previous knowledge about this region, I intend to audit the following issues:  
- Are the cities names correct?
- Are the street names correct?
- Are there abbreviations?
- Are the postal codes consistent?  

It must be said that here the data are being first explored iteratively. Besides it is recommended to have one script for each field that is being audited, the whole process will be done through this Jupyter notebook in order to give an overview of the cleaning process. At the end, the code will be transferred to a standalone Python script (.py), in order to facilitate its automation when converting, cleaning and exporting data to a MongoDB collection, for example. 

### Are the cities names correct?
In order to answer this question, I need first to know where this information is in the OpenStreetMaps (OSM) data model, which can be found in [5]. Consulting the documentation we get to know we are looking for the *addr:city* key. 

In [103]:
#%% Finding the cities in the dataset
def list_cities(filename):
    cities = []
    for _, elem in ET.iterparse(filename):
        if elem.tag == 'tag':
            k = elem.attrib['k']
            v = elem.attrib['v'].lower()  #Lowering the uppercase text
            if k == 'addr:city':
                if v not in cities:
                    cities.append(v)
    print('There are {0} distinct cities in the dataset.'.format(len(cities)))
    return cities

cities = list_cities(FILENAME)
print(cities)

There are 18 distinct cities in the dataset.
['santa rosa', 'condor', 'ijuí', 'panambi', 'santo ângelo', 'três de maio', 'panambi - rs', 'santo cristo', 'eugênio de castro', 'santo augusto', 'santo angelo', 'cruz alta', 'vila sírio', 'cerro largo', 'são josé do mauá', 'são miguel das missões', 'horizontina', 'ijui']


#### (1) Same cities are recorded with distinct names due to hyphenization or accentuation
Even though I choose to use lowercase text, there are cities whose names are written with accentuation or hyphenized with the State abbreviation. A possible way to fix it is mapping the correct name to each case:

In [104]:
#%% Cleaning the cities names:
expected_cities = ['santa rosa', 'condor', 'ijuí', 'panambi', 'santo ângelo', 'três de maio',
            'santo cristo', 'eugênio de castro', 'santo augusto', 'cruz alta', 'vila sírio', 
            'cerro largo', 'são josé do mauá', 'são miguel das missões', 'horizontina']

In [105]:
def audit_cities(expected_cities, filename):
    weird = []
    for _, elem in ET.iterparse(filename):
        if elem.tag == 'tag':
            k = elem.attrib['k']
            v = elem.attrib['v'].lower()  #Lowering the uppercase text
            if k == 'addr:city':
                if v not in expected_cities:
                    weird.append(v)
    weird = set(weird)
    print('There are {0} not expected cities in the dataset.'.format(len(weird)))
    return weird

In [106]:
audit_cities(expected_cities, FILENAME)

There are 3 not expected cities in the dataset.


{'ijui', 'panambi - rs', 'santo angelo'}

### Are the street names correct? Are there any abbreviation?
We will now iterate over all the registers to find wrong street names or abbreviations.

In [107]:
expected = ["Rua", "Avenida", "Praça", "Via", "Estrada", "Travessa", "Linha", "Alameda", "Largo", "Parque", "Rodovia"]

### IMPORTANT: Brazilian street types are in the beginning of the phrase:
street_type_re = re.compile(r'^\b\S+\.?', re.IGNORECASE)

In [108]:
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)

In [109]:
def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

In [110]:
def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])

    return street_types

In [111]:
def test_streets():
    st_types = audit(FILENAME)
    pprint.pprint(dict(st_types))
    
test_streets()

{'14': {'14 de Julho'},
 '15': {'15 de Novembro'},
 'Av.': {'Av. Gustav Kuhlmann', 'Av. Santa Bárbara'},
 'BR': {'BR 285'},
 'BR-285': {'BR-285'},
 'BR-392': {'BR-392'},
 'BR158': {'BR158'},
 'Dom': {'Dom Pedro II'},
 'ERS-342': {'ERS-342'},
 'Getúlio': {'Getúlio Vargas'},
 'Padre': {'Padre Afonso Rodrigues'},
 'Paulo': {'Paulo Klemann'},
 'RS': {'RS 218'},
 'Santa': {'Santa Lucia'}}


From the 'weirdos' found above, we could write some mapping dictionaries and functions to clean the data.

### Are the postal codes consistent?
The Brazilian postal codes are known as CEP and must contain 8 digits in the following format '00000-000'. Furthermore, in the Missões region the CEP numbers always start with the digit 9. 

In [112]:
cep = re.compile(r"[0-9]{5}-[0-9]{3}") #Alternative: cep = re.compile('d{5}-d{3}')

In [113]:
def is_cep(elem):
    return (elem.attrib['k'] == "addr:postcode")

In [114]:
def audit_cep(filename):
    osm_file = open(filename, "r")
    bad_cep = []
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_cep(tag):
                    v = tag.attrib['v']
                    if cep.match(v):
                        pass
                    else:
                        bad_cep.append(v)
    return bad_cep

In [115]:
audit_cep(FILENAME)

['98910000']

From the questions and the attemptives to answer them, we could create some mapping dictionaries and updating functions to clean the data. They are suppresed from this notebook but can be found in the *final_project_code.py* script.

--------------
## Cleaning, shaping and exporting data
After auditing and deciding which data must be cleaned, it's time to shape the data into the model that will be exported to a MongoDB collection.  

The output should be a list of dictionaries that look like this:
>*{  
"id": "2406124091",  
"type: "node",  
"visible":"true",  
"created": {  
          "version":"2",  
          "changeset":"17206049",  
          "timestamp":"2013-08-03T16:43:42Z",  
          "user":"linuxUser16",  
          "uid":"1219059"  
        },  
"pos": [41.9757030, -87.6921867],  
"address": {  
          "housenumber": "5157",  
          "postcode": "60625",  
          "street": "North Lincoln Ave"  
        },  
"amenity": "restaurant",  
"cuisine": "mexican",  
"name": "La Cabana De Don Luis",  
"phone": "1 (773)-271-5176"  
}*

After concluding the audit and cleaning process, all the functions were concatenated on a python script in order to export the final data to a json file (Missoes.osm.json), which then can be loaded to a MongoDB collection.

-------------------
## Data exploration through MongoDB API

In [120]:
## Loading MongoDB libraries
from pymongo import MongoClient
import pprint
## Loading Pandas for data analysis
import pandas as pd

In [121]:
## Acessando o banco de dados
client = MongoClient('mongodb://localhost:27017/')

In [122]:
## Lendo os bancos de dados disponíveis:
client.database_names()

/home/tbnsilveira/anaconda3/envs/analytics3/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: database_names is deprecated. Use list_database_names instead.
  from ipykernel import kernelapp as app


['admin', 'config', 'examples', 'local', 'udacity']

In [123]:
## Lendo as coleções disponíveis:
client.udacity.collection_names()

/home/tbnsilveira/anaconda3/envs/analytics3/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  from ipykernel import kernelapp as app


['osm']

### Exploring the database

In [144]:
## Creating an alias:
db = client.udacity.osm

In [145]:
## Checking the first instance:
firstInstance = db.find_one()
pprint.pprint(firstInstance)

{'_id': ObjectId('5badbef1ef1c25cdb2e4eefb'),
 'created': {'changeset': '23072634',
             'timestamp': '2014-06-21T23:33:37Z',
             'uid': '1714450',
             'user': 'Papibaquígrafo',
             'version': '2'},
 'id': '282841128',
 'pos': [-28.1743824, -54.8258869],
 'type': 'node'}


Once the data is available, now it's time to explore it in the way to find the answers for some questions:

### Database overview
We have collected a small portion of OSM. Knowing the data is written in "nodes" and "ways", it would be important to know quantitavely the amount of data available, which in turn may determine the confidence of any conclusion taken from this data. 

In [146]:
def data_information():
    nDocs = db.find().count()
    nNodes = db.find({'type':'node'}).count()
    nWays = db.find({'type':'way'}).count()
    nUsers = len(db.distinct("created.user"))
    print('Database overview:\nDocuments: {0}\nNodes: {1}\nWays: {2}\nUsers: {3}'.format(nDocs, nNodes, nWays, nUsers))

In [147]:
data_information()

/home/tbnsilveira/anaconda3/envs/analytics3/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  from ipykernel import kernelapp as app
/home/tbnsilveira/anaconda3/envs/analytics3/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  app.launch_new_instance()
/home/tbnsilveira/anaconda3/envs/analytics3/lib/python3.6/site-packages/ipykernel/__main__.py:4: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.


Database overview:
Documents: 425642
Nodes: 380322
Ways: 45320
Users: 188


-------------------
### Do the database contains all the 46 oficially current cities of the Missões region?

We already know, from the audit and cleaning process, that the collected region has  **only 15 cities** (take a look at the "expected cities" in the audit section). This is due to the small portion of data selected to export from OSM, which does not mean the data for the other cities are not available.  
For learning purpose, let's suppose it wasn't me who audited and cleaned this dataset, i.e. I just had access to this MongoDB collection. If it was the case I would run the following code in order tho find which cities are in the dataset:

In [148]:
cities = db.aggregate([
    {'$match':{'address.city':{'$exists':1}}},
    {'$group':{'_id':'$address.city','count':{'$sum':1}}},
    {'$sort':{'count':-1}}
])

In [149]:
list(cities)

[{'_id': 'ijuí', 'count': 19},
 {'_id': 'cerro largo', 'count': 19},
 {'_id': 'panambi', 'count': 14},
 {'_id': 'santa rosa', 'count': 12},
 {'_id': 'condor', 'count': 6},
 {'_id': 'eugênio de castro', 'count': 6},
 {'_id': 'vila sírio', 'count': 5},
 {'_id': 'três de maio', 'count': 5},
 {'_id': 'santo ângelo', 'count': 4},
 {'_id': 'são josé do mauá', 'count': 3},
 {'_id': 'santo cristo', 'count': 2},
 {'_id': 'são miguel das missões', 'count': 1},
 {'_id': 'santo augusto', 'count': 1},
 {'_id': 'horizontina', 'count': 1},
 {'_id': 'cruz alta', 'count': 1}]

--------------------
### What are the amenities and cuisines registered for this region?
The Missões region is known as a historical and touristic place. The Jesuit Missions of the Guaranis, and specifically the ruins of São Miguel, are considered a world heritage by UNESCO since 1983 [6]. In this way, and knowing that amenities and cuisines are closely related to touristic regions, it would be interesting to explore how this services are operating in this region -- or at least, how this information is being inputed in the OpenStreetMap.

In [166]:
def find_amenities():
    result = db.aggregate([
        {"$match":{"amenity":{"$exists":1},"type":"node"}},
        {"$group":{"_id":"$amenity","count":{"$sum":1}}},
        {"$sort":{"count":-1}}])
    return list(result)

In [151]:
find_amenities()

[{'_id': 'place_of_worship', 'count': 90},
 {'_id': 'school', 'count': 63},
 {'_id': 'fuel', 'count': 62},
 {'_id': 'townhall', 'count': 23},
 {'_id': 'bank', 'count': 22},
 {'_id': 'police', 'count': 22},
 {'_id': 'restaurant', 'count': 20},
 {'_id': 'hospital', 'count': 19},
 {'_id': 'bus_station', 'count': 17},
 {'_id': 'parking', 'count': 15},
 {'_id': 'clinic', 'count': 10},
 {'_id': 'bar', 'count': 10},
 {'_id': 'ferry_terminal', 'count': 10},
 {'_id': 'pharmacy', 'count': 7},
 {'_id': 'post_office', 'count': 6},
 {'_id': 'college', 'count': 5},
 {'_id': 'fast_food', 'count': 5},
 {'_id': 'ice_cream', 'count': 4},
 {'_id': 'pub', 'count': 4},
 {'_id': 'telephone', 'count': 4},
 {'_id': 'arts_centre', 'count': 4},
 {'_id': 'taxi', 'count': 4},
 {'_id': 'courthouse', 'count': 4},
 {'_id': 'car_repair', 'count': 3},
 {'_id': 'community_centre', 'count': 3},
 {'_id': 'university', 'count': 2},
 {'_id': 'bbq', 'count': 2},
 {'_id': 'charging_station', 'count': 2},
 {'_id': 'marketplac

In [152]:
def find_amenities_byCity():
    result = db.aggregate([
        {'$group': {'_id': '$address.city',
                    'amenities': {
                        '$addToSet': '$amenity'},
                    'cuisine': {
                        '$addToSet': '$cuisine'}
                    }}])
    return list(result)

And how are these amenities and cuisines distributed among the cities?

In [153]:
data = pd.DataFrame(find_amenities_byCity())
data

,_id,amenities,cuisine
0,são miguel das missões,[],[]
1,são josé do mauá,"[community_centre, fuel]",[]
2,horizontina,[],[]
3,cerro largo,"[place_of_worship, restaurant, bank, universit...",[regional]
4,cruz alta,[],[]
5,ijuí,"[bank, library, university, arts_centre, polic...",[]
6,None,"[food_court, toilets, swimming_pool, courthous...","[steak_house, fine_dining, italian, pizza, bur..."
7,vila sírio,"[fuel, community_centre]",[]
8,santa rosa,"[college, school, townhall, place_of_worship]",[]
9,panambi,"[pharmacy, fuel, restaurant, college]","[Churrascaria, pizza]"


-----------
### Additional suggestions
Besides there are reach information available in OpenStreetMap database, being one that knows that region pretty well I could say there are so many information that should be added to this awesome open project. In this way, I will add up the following suggestions:  

#### (1) Enrich OSM with open data from oficial sources:  
Brazil is a country with a relatively high rate of open data. The IBGE - Brazilian National Institute for Geography and Statistics provides open information for all that region, including service providers registers, which can be found in www.ibge.gov.br. 

#### (2) Improve the available information in order to promote tourism in the region:
There are some domestic airports in the region and an international one. However, these amenities are not foreseen in the OSM amenity set of keys. That's the same case for hotels. 

#### (3) Improving data quality:  
As anounced in the OSM Wiki [7], the 'yes' tag should be verified in order to be tagged with another value. Also, during the data exploration I found some amenities without address information, which could be added to it.

-----------
## Drawing some conclusions

Despite the tourist and historical appeal of the place, there are a few scarce facilities in the region. There is no record of even an airport or bus station that serves the region.
Initially, I attribute the fact to perhaps a small number of employees. However, the data show 188 employees in the region -- a relatively satisfactory number.  

This analysis shows the opportunity to foster community participation in the data collection of this region, as well as opens up the opportunity for further investigation into these facts.

From a technical point of view, this simple project allows the consolidation of the data wrangling concepts, including both the theoretical and practical approaches that involve the processes of data collection, auditing, cleaning and exporting -- fundamental steps in the work of any data scientist.

----------------
## References
[1] https://en.wikipedia.org/wiki/Miss%C3%B5es  
[2] https://en.wikipedia.org/wiki/Spanish_missions_in_South_America  
[3] https://jasonicarter.github.io/openstreetmap-data-wrangling-mongodb/  
[4] https://eberlitz.github.io/2015/09/18/data-wrangle-openstreetmaps-data/  
[5] https://wiki.openstreetmap.org/wiki/Key:addr  
[6] http://www.unesco.org/new/en/brasilia/culture/world-heritage/list-of-world-heritage-in-brazil/jesuit-missions-of-the-guaranis/  
[7] https://wiki.openstreetmap.org/wiki/Key:amenity#Tags_that_need_improvement